In [14]:
import pandas as pd
data = pd.read_csv('fifa_players.csv')
pd.set_option('display.max_rows', None)  # Display all rows


C:\Users\chinu\AppData\Local\Temp\ipykernel_37104\1472952389.py:2: DtypeWarning: Columns (108) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('fifa_players.csv')


In [15]:
data = data[data['fifa_version'] == 24.0]
columns_to_drop = ['player_url', 'fifa_version', 'fifa_update', 'update_as_of', 'short_name', 'long_name', 'dob', 'club_name', 'league_id',
                   'league_level', 'club_position', 'club_loaned_from', 'club_joined_date', 'club_contract_valid_until_year', 
                   'nationality_id', 'nation_team_id', 'nation_position', 'nation_jersey_number', 'international_reputation', 'real_face',
                   'release_clause_eur', 'player_tags', 'player_traits', 'pace', 'shooting', 'passing', 'dribbling', 'defending', 'physic', 
                   'goalkeeping_speed', 'ls', 'rs', 'rf', 'lf', 'ram', 'lam', 'ldm', 'lcm', 'rdm', 'rcm', 'lcb', 'rcb']
data= data.drop(columns=columns_to_drop)

In [16]:
null_columns = ['value_eur', 'wage_eur', 'club_team_id', 'league_name', 'club_jersey_number']
data = data.dropna(subset=null_columns)

In [17]:
def extract_positions(positions):
    if pd.isna(positions):
        return None, None
    positions_list = positions.split(', ')
    first_position = positions_list[0]
    alternative_positions = positions_list[1:] if len(positions_list) > 1 else []
    return first_position, alternative_positions
data[['first_position', 'alternative_positions']] = data['player_positions'].apply(lambda x: pd.Series(extract_positions(x)))
data = data.drop(columns='player_positions')

In [18]:
data['first_position'].unique()

array(['ST', 'CM', 'CF', 'GK', 'LW', 'CDM', 'CB', 'RW', 'CAM', 'RB', 'RM',
       'LB', 'LM', 'LWB', 'RWB'], dtype=object)

In [19]:
import re
def convert_expression(value):
    try:
        if isinstance(value, str):
            # Handle expressions with + or -
            if '+' in value or '-' in value:
                return eval(value)
            elif re.match(r'\d+', value):
                return int(value)
        else:
            return value
    except ValueError:
        return value
columns_to_convert = ['st','lw', 'cf', 'rw', 'cam', 'lm', 'cm', 'rm', 'lwb', 'cdm', 'gk', 'cb', 'rb', 'lb', 'rwb']
for column in columns_to_convert:
    data[column] = data[column].apply(convert_expression)
for column in columns_to_convert:
    data[column] = data[column].astype(int)


In [20]:
# List of columns to one-hot encode
columns_to_one_hot_encode = ['preferred_foot', 'work_rate', 'body_type']

# Apply one-hot encoding to the specified columns
data_encoded = pd.get_dummies(data, columns=columns_to_one_hot_encode)

for column in data_encoded.columns:
    if data_encoded[column].dtype == 'bool':
        data_encoded[column] = data_encoded[column].astype(int)


In [21]:
data_encoded.info()

<class 'pandas.core.frame.DataFrame'>
Index: 18250 entries, 0 to 18349
Data columns (total 86 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   player_id                    18250 non-null  int64  
 1   overall                      18250 non-null  int64  
 2   potential                    18250 non-null  int64  
 3   value_eur                    18250 non-null  float64
 4   wage_eur                     18250 non-null  float64
 5   age                          18250 non-null  int64  
 6   height_cm                    18250 non-null  int64  
 7   weight_kg                    18250 non-null  int64  
 8   club_team_id                 18250 non-null  float64
 9   league_name                  18250 non-null  object 
 10  club_jersey_number           18250 non-null  float64
 11  nationality_name             18250 non-null  object 
 12  weak_foot                    18250 non-null  int64  
 13  skill_moves          

In [22]:
data_encoded.to_csv('fifa_players_processed.csv')